# Lecture 04 Self-Attention

#### 1 Basic Self-Attention Implementation from Scratch

This example demonstrates the core concept of self-attention without relying on any deep learning libraries. It computes the attention scores and outputs for a simple input sequence.

In [9]:
import numpy as np

def softmax(x):
    """Compute softmax values for each set of scores in x."""
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))

    return e_x / e_x.sum(axis=-1, keepdims=True) 
    # probabilities along the row equal to 1

1. Subtracting the Maximum Value
    - Purpose: Improve numerical stability to prevent potential overflow issues when computing exponentials.

2. The axis parameter ```axis=-1``` specifies the dimension along which to perform the operation. In NumPy, negative integers can be used to refer to axes from the end. Here, axis=-1 refers to the last axis of the array. For example:
    - If x is a 2D array (matrix) with shape (m, n), axis=-1 is equivalent to axis=1, which operates along the columns.
    - If x is a 3D array with shape (a, b, c), axis=-1 refers to the c dimension. 

3. The last line ```e_x / e_x.sum(...)``` divides each exponential by the sum of exponentials along the specified axis (this this case, along the row), resulting in probabilities that sum to 1.

In [11]:
def self_attention_basic(inputs):
    """
    Compute self-attention for input vectors.

    Args:
        inputs: A NumPy array of shape (sequence_length, embedding_dim)
    
    Returns:
        output: Self-attended output of shape (sequence_length, embedding_dim)
    """
    # Initialize weight matrices (for simplicity, using identity matrices)
    # inputs.shape[1] is embed_dim, i.e. d_model 
    W_q = np.eye(inputs.shape[1])
    W_k = np.eye(inputs.shape[1])
    W_v = np.eye(inputs.shape[1])

    # Compute queries, keys, and values
    Q = inputs @ W_q  # (seq_len x d_model) x (d_model x d_model), d_model is embed_dim
    print("The query Q is:\n", Q)
    K = inputs @ W_k  # (seq_len x d_model)
    print("The key K is:\n", K)
    V = inputs @ W_v  # (seq_len x d_model)
    print("The value (meaning) V is:\n", V)
    
    # Compute attention scores
    scores = Q @ K.T # (seq_len, seq_len)
    print("The attention score is:\n", scores)
    # Scaled Dot-Product Attention: scores = np.dot(Q, K.T) / np.sqrt(inputs.shape[1])  # (seq_len, seq_len)
    attention_weights = softmax(scores)  # (seq_len, seq_len)
    print("The attention weights to be applied on value (meaning) is:\n", attention_weights)
    
    # Compute the weighted sum of values
    output = attention_weights @ V  # (seq_len, embed_dim)
    return output

#### Example 1

In [12]:
if __name__ == "__main__":
    # Sample input: 3 tokens with embedding size 4
    inputs = np.array([
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [1, 1, 1, 1]
    ])

    output = self_attention_basic(inputs)
    np.set_printoptions(suppress=True, precision=8)
    print("Self-Attention Output:\n", output)

The query Q is:
 [[1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 1. 1. 1.]]
The key K is:
 [[1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 1. 1. 1.]]
The value (meaning) V is:
 [[1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 1. 1. 1.]]
The attention score is:
 [[2. 0. 2.]
 [0. 2. 2.]
 [2. 2. 4.]]
The attention weights to be applied on value (meaning) is:
 [[0.46831053 0.06337894 0.46831053]
 [0.06337894 0.46831053 0.46831053]
 [0.10650698 0.10650698 0.78698604]]
Self-Attention Output:
 [[0.93662106 0.53168947 0.93662106 0.53168947]
 [0.53168947 0.93662106 0.53168947 0.93662106]
 [0.89349302 0.89349302 0.89349302 0.89349302]]


**Explanation:** <br>
- Input Representation: The input is a sequence of vectors (e.g., word embeddings). In this example, we have a sequence length of 3 with embedding dimensions of 4.
- Weight Matrices: For simplicity, the weight matrices W_q, W_k, and W_v (used to compute queries, keys, and values) are initialized as identity matrices. In practice, these are learned during training.
- Queries, Keys, and Values: Compute queries (Q), keys (K), and values (V) by multiplying the input with the respective weight matrices.
- Attention Scores: Calculate the attention scores by taking the dot product of Q and the transpose of K (in Transformer, you need to scale the product by the square root of the embedding dimension to stabilize gradients.)
- Softmax: Apply the softmax function to obtain attention weights, which indicate the importance of each token in the sequence relative to others.
- Weighted Sum: Multiply the attention weights with the values V to get the final self-attended output.

#### Example 2 Jiraiya Code

<img src="Jiraiya_Code.jpg" width="600">

In [13]:
# Example 2 Jiraiya Code
if __name__ == "__main__":
    # Sample input: 3 tokens with embedding size 3
    inputs = np.array([
        [9, 31, 8],
        [106, 7, 0],
        [207, 15, 0]
    ])

    output = self_attention_basic(inputs)
    np.set_printoptions(suppress=True, precision=8)
    print("Self-Attention Output:\n", output)

The query Q is:
 [[  9.  31.   8.]
 [106.   7.   0.]
 [207.  15.   0.]]
The key K is:
 [[  9.  31.   8.]
 [106.   7.   0.]
 [207.  15.   0.]]
The value (meaning) V is:
 [[  9.  31.   8.]
 [106.   7.   0.]
 [207.  15.   0.]]
The attention score is:
 [[ 1106.  1171.  2328.]
 [ 1171. 11285. 22047.]
 [ 2328. 22047. 43074.]]
The attention weights to be applied on value (meaning) is:
 [[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
Self-Attention Output:
 [[207.  15.   0.]
 [207.  15.   0.]
 [207.  15.   0.]]


In [20]:
x = [1106,  1171,  2328]
print('To improve numerical stability, x - np.max(x) is: ', x - np.max(x, axis=-1, keepdims=True))
e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
print('The softmax score is: ', e_x)

To improve numerical stability, x - np.max(x) is:  [-1222 -1157     0]
The softmax score is:  [0. 0. 1.]
